# <img style="float: left; padding-right: 10px; width: 45px" src="img/summer_school.png"> S-109A Introduction to Data Science 


## Lab 5: Ensembling methods, and Neural Networks

**Harvard University**<br>
**Summer 2021**<br>
**Instructor:** Kevin Rader<br>
**Notebook Authors:** Will Claybaugh, David Sondak, Kevin Rader, Chris Gumb

---

## <font color='red'> Run the cell below to properly highlight the exercises</font>

In [ ]:
from IPython.display import HTML
style = "<style>"+\
    "div.exercise { background-color: #bf6cf1;border-color: #E9967A; border-left: 5px solid #800080; padding: 0.5em;}"+\
    "div.discussion { background-color: #90EE90;border-color: #E9967A; border-left: 5px solid #800080; padding: 0.5em;}"+\
    "</style>"
HTML(style)

## Learning Goals
After this lab, you should be able to:
- Discuss when combining several models could help improve overall accuracy
- Tune across multiple hyperparameters using cross_val_scores, GridSearchCV, or ParamGrid
- Explain why boosting and bagging are particularly effective versions of ensembling

In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from IPython import display
%matplotlib inline

## Part 0: Jellybeans?
<img src='img/jelly_beans.jpg' style='width:200px;'>

Imagine we'd brought a jar of jelly beans to lab today and asked everyone to guess how many are in the jar. Whoever guesses closest to the exact number of jelly beans gets to keep them all! (If several people are the same distance from the truth, they split the jar equally.)

In this example, the members of the class play the role of a single model/estimator, and by combining the class's predictions we get a more accurate overall estimate. When we combine multiple models, it's called _ensembling_. 

We can learn a lot about ensembling by imagining a group of people all guessing how many beans are in a jar. What features would we want those people to have? What biases or pathologies could ruin the overall ensemble?

<div class="discussion"><b>Discussion</b></div>

**Q0.1** What happens to the combined guess if each member of the class makes the same kind of mistakes?

*your answer here*

**Q0.2** What happens if the class is all clones and all make the same guess?

*your answer here*

**Q0.3** What happens if a few classmembers are really, really, really bad at guessing?

*your answer here*

**Q0.4** What happens if people are influenced by the previous guess, or by peer pressure?

*your answer here*

**Q0.5** Are there better combination rules than 'take the average'?

*your answer here*

**Q0.6** What happens if there are multiple jars in which each classmate provides a separate guesss for?  Is there an issue if people who guess too high on one jar tend to guess too high on another?  What if guessing too high on one means you'll likely guess too low on the other?

*your answer here*

**Q0.7** How should we deal with people who hate jelly beans, and purposefully guessed wrong about that jar?

*your answer here*

**Q0.8** Putting it all together, what would a good group of estimators look like? What would a good combination rule look like?

*your answer here*

<hr>

## Part 1: Combining Models
The data today are from https://www.kaggle.com/hobako1993/sp1-factor-binding-sites-on-chromosome1/home:

>This dataset includes SP1 transcription factor binding and non-binding sites on human chromosome1. It can be used for binary classification tasks in bioinformatics. There are 1200 sequences for binding sites (BS) and 1200 sequences for non-binding sites (nBS). We have labeled sequences with 1 for BS and 0 for nBS. Each sequence is 14 nucleobase length, which is converted to numeric string using codes below, assigned to each nucleobase 00 for A 01 for T 10 for C 11 for G

In [ ]:
df_train = pd.read_csv('data/genes_train.csv', index_col=0)
df_test = pd.read_csv('data/genes_test.csv', index_col=0)

df_train.head()

#### The tuning set
In addition to the usual train/test split above, we further split our training data into 
1. A true training set, and 
2. A held-out tuning set which we'll use to learn rules that combine base models.

|training set|tuning set|test set|
|--|--|--|
|Build lots of models, including tuning their hyperparameters by doing CV within these data| Learn how to combine base models so that the combination better predicts data the models haven't seen| Find out how good the models+combination is on data it hasn't seen

The overall pipeline would look like this
1. Fit models on the training data, finding hyperparameters via CV if needed
2. Have each model make a prediciton on the tuning data
3. Learn how to best combine the models' tuning predictions. For instance, fit a linear regression with the model predictions as the input and the true y values as the output
4. Evaluate the models+combination rule on the test data:
    1. Have each model predict each test point
    2. Use the rule/model from 3 to combine the predictions into a single prediciton for each point
    3. Compare the predictions above to the true y values
5. In production, each model makes prediciton on the new data, and we use the rule/model from 3 to combine those into a final prediction

In [ ]:
from sklearn.model_selection import train_test_split

# further split the training data into training and tuning sets
df_train, df_tune = train_test_split(df_train, test_size=.2, random_state=8)

#### Design matrices
In each set, we build the design matrix and the target

In [ ]:
def get_response_and_design(df):
    design = df.iloc[:, df.columns != 'label']
    response = df['label'].values
    
    return response, design

y_train, X_train = get_response_and_design(df_train)
y_tune, X_tune = get_response_and_design(df_tune)
y_test, X_test = get_response_and_design(df_test)

## Ensembling
In lecture, we've already discussed how boosting and bagging build several base models and then combine them. Ensembling is the general practice of combining individual base models in order to get a more accurate overall estimate. In a sense, ensembling builds a meta-model that combines predictions from original base models.

**The two pieces of any ensemble are:**  
**1. The original base models**  
**2. The rule for combining the base models' predictions**
** **

<div class="discussion"><b>Discussion</b></div>
<br>

**Q1.1** What are Bagging and Boosting's methods of building esnemble models?  What combination rules do they use?

**Answer**:


**Q1.2** What would Bagging's models look like as a room full of people? What would Boosting's models look like as a room full of people?

**Answer**:


### Building our ensemble
Here, we'll take the original base models as given. Each base model in the array below has been trained on the training dataset.

For those who haven't seen the `.joblib` format before, it's SKLearn's preferred method for saving modeles to disk. 
You can read more here: https://scikit-learn.org/stable/modules/model_persistence.html


In [ ]:
# You'll have to install `joblib` if you don't have it already
!pip install joblib

In [ ]:
import joblib

# load all our saved models into a list
models = []
with open('data/models.joblib', 'rb') as f:
    while True:
        try:
            models.append(joblib.load(f))
        except EOFError:
            break

Individually, the models are rather poor, and don't even beat a logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

# get the accuracy score from logistic regression
# cross validation to determine the best regularization parameter, C
LR_score = LogisticRegressionCV().fit(X_train, y_train).score(X_test,y_test)

scores = []
for cur_model in models:
    scores.append(cur_model.score(X_test,y_test))

fig, ax = plt.subplots(1,1,figsize=(20,10))
ax.hist(scores,20, label="Decision Tree Base models");
ax.axvline(LR_score, color='red',label="Logistic Regression")
ax.set_xlabel('Accuracy Scores', fontsize=24) # DLS:  Added x-label
ax.set_ylabel("Number of Models at Each Score")
ax.legend(loc='best', fontsize=24)
plt.show()

To summarize the models, we have data frames recording each model's prediction on each point in the training/tuning/test set.

The 'tuning' set below is new. As mentioned, we need a set of data that isn't the training data, nor used to set hyperparameters in the original models (so, not the validation set), nor the final test data. We'll use this data in just a bit

In [ ]:
predictions_train = pd.read_csv("data/predictions_train.csv", index_col=0)
predictions_tune = pd.read_csv("data/predictions_tune.csv", index_col=0)
predictions_test = pd.read_csv("data/predictions_test.csv", index_col=0)

predictions_tune.head()

Considering the first row above, we see that model0 predicted the first tuning point to be class 0, model1 disagreed and thought it was class 1, and most of the other models think it's class 0. Other rows tell us about other observations in the tuning set.

### Combining the models
Perhaps the simplest way of combining the models' predictions (above) is a majority vote. Let's compute the test score under this rule

In [ ]:
# axix = 1: get the mean along a row
meta_test_predictions = np.mean(predictions_test,axis=1) > 0.5  #do we have more than 50% 1s?
print("Test accuracy (Classify by majority vote): ", accuracy_score(y_test, meta_test_predictions))

That's quite a bit better! The majority vote is scoring far better than any individual model.

**Note**:
- Working with probabilities is more information-rich. Especially in the two-class setting, predictions take probabilities like .51 and pretend that they're actually 1.0. You'll deal more with ensembling via probabilities on your homework. For now, we'll stick with the models' predictions.
- The Receiver Operating Characteristic Area Under the Curve (ROC AUC) is generally considered a more reliable metric than accuracy for a classification problem. Feel free to also use <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html" target="_blank">roc_auc(y_test, y_proba)</a> to score the models in this notebook.

** **

<div class="discussion"><b>Discussion</b></div>

**Q1.3** Suppose each modelling process reflected in the ensemble has the same bias and variance. How does majority voting affect combined bias and variance?

**Answer**:


**Q1.4** Can you think of a weakness in majority voting? (Recall the original models that had less than 50% accuracy...)

**Answer**:


### Complex combination rules
Let's try giving more weight to the models that are performing better. We already have the models' test-set performance in `scores` so let's use that... 

But wait! We would be using test data to (ultimately) help predict test data, and invalidate our model. And weighting via the training set performance would just say "listen to the most overfit model you can find, that guy's a genius." 

This is where the tuning set we left aside comes into play. It gives us a chance to measure how well each component model does on new data, without spoiling the test set. (If we had valid estimates of how well each model does out-of-sample, e.g. from out-of-bag estimates, we could use those to decide weights)

If we hadn't left a tuning set aside, we'd have to go back to the very beginning, set aside part of our training set to be the tuning set, and fit the base models to just the reduced training set.

**Speaker note**: point out that we've got the old one-tuning-set problem, and CV can assist

In [ ]:
# get each model's overall performance on the tuning data
tuning_scores = [m.score(X_tune, y_tune) for m in models]

# divide by the sum so the weights all sum to 1.
weights = np.array(tuning_scores)/np.sum(tuning_scores)

print("First five weights:", weights[0:5])

# # on each tuning observation, apply the weight assigned to each model and predict as 1
# # if the weighted majority breaks 50%
# # np.dot performs matrix multiplication
weighted_predictions = np.dot(predictions_test, weights) > 0.5

test_acc = accuracy_score(y_test,weighted_predictions)
print(f'Test Acc: {test_acc:.3f}')

It turns out that weighting didn't change accuracy much. Though, to be fair, the weights are all basically .01, which is what they would be for a pure majority vote. Although we could spend time finding the right way to convert accuracy scores to weights, what we'd really like is to _learn the weights and threshold that are optimal for correctly classifying new points_. [Where have we heard _that_ before?] 

<div class="exercise"><b>Exercise</b></div>

**Q1.5** Create a logistic regression metamodel with ridge regularization to find weights for each of the 100 original models. You should use cross validation to tune the regularization.

In [ ]:
# learn the optimal weights and intercept
# (intercept is related to whether we need a 50% majority or a 66% majority or an 80% majority)
# your code here


**Q1.6** Combine the models using the weights you just found.

**Hint:** Don't over think it!

In [ ]:
# use weights and intercept to combine the test data predictions
# your code here

**Q1.7** Report the accuracy of the metamodel's predictions. How does it perform compared to the individual estimators and the the previous metamodels? What do you think explains this performance?

In [ ]:
# evaluate logistic metamodel
# your code here


**Answer:**


### More Information?

Up to this point we've been trying to find ways of combining the original models' _class predictions_ to improve our accuracy. But what we'd _really_ like to use are the models' _probability predictions_. This would give us much more information to work with.It would be important to know how confident each model is in its predictions.

In [ ]:
def df_from_model_probas(models, X):
    '''
    Builds a Dataframes of probability predictions on X from a list of classifiers
    '''
    model_probas = [{f'model{i}': m.predict_proba(X)[:,1]} for i,m in enumerate(models)]
    df = pd.concat((pd.DataFrame(m) for m in model_probas), axis=1)
    return df

probas_train = df_from_model_probas(models, X_train)
probas_tune = df_from_model_probas(models, X_tune)
probas_test = df_from_model_probas(models, X_test)

probas_tune.head()

**Q1.8** Fit a 2nd logistic metamodel, this time on the probability predictions. How does it perform?

In [ ]:
# learn the optimal weights and intercept
# (intercept is related to whether we need a 50% majority or a 66% majority or an 80% majority)
# your code here


**Answer:**


We'll use the probabilities for training our metamodels from here on. To keep things simple we'll rename the dataframes to go back the the `predictions_...` convention. You can always experiment with using the class labels on the rest of the notebook by setting `USE_PROBAS` to False and re-running.

In [ ]:
USE_PROBAS = True
if USE_PROBAS:
    predictions_tune = probas_tune
    predictions_train = probas_train
    predictions_test = probas_test

**Q1.8** How well does a decision tree do at combining the models? (Be careful about how you pick your parameters. Again, they should be tuned with cross validation)

**Hint:** There is no CV version of the decision tree in SKLearn like there is for logistic regression. You'll need to use something like `cross_val_scores` instead.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
# your code here


**Answer:**


**Return to the main Zoom room** :) 

Importantly, any parameter hunting we do for the meta-model needs to be done via data from the tuning set. We're not allowed to peek at the test set, and the training set has already been used to tune and fit the base models. *All meta-parameters are found via the tuning set*.

**Speaker note**: talk about building the models yourself, e.g. combining well-tuned RF to get a good combined model 

### Summary of Ensembling
- Ensemble models can be understood as building a bunch of models on the training data and combining them somehow
- Visually, you can imagine a dataset consisting of each model's prediction or (if available) its probability estimate for each data point
- The art of ensembling is in building models that complement each other, and picking a rule to combine them
- You can even use the data to tell you what the combination rule should be, but you have to use data separate from the training and test sets to learn this rule. Simply combining the models via linear/logistic regression is a popular choice.

## Part 2: Bagging & Hyperparameter Tuning
You've been working with bootstrapping since the beginning of the course, so you should be a pro when it comes to bagging. Remember that random forests are a generalization of bagged trees.



### Tuning the hyperparameters

Random Forests generally perform very well out-of-the-box, with the sklearn's default hyperparameters. Some of the tunable parameters are:
- The number of trees in the forest: `n_estimators`, int, default=100
- The complexity of each tree: 
    - stop when a leaf has <= `min_samples_leaf` samples 
    - the `max_depth` of each tree
- The sampling scheme: number of features to consider at any given split: `max_features` {“auto”, “sqrt”, “log2”}, int or float, default=”auto”.

#### Hints:

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html" target="_blank">RandomForestClassifier()</a> : Defines the RandomForestClassifier and includes more details on the definition and range of values for its tunable parameters.

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict_proba" target="_blank">model.predict_proba(X)</a> : Predict class probabilities for X

### Vanilla random forest

We'll start by training a Random Forest Classifier using the default parameters and calculate the test accuracy.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
seed = 0
# Define a Random Forest classifier with randon_state = seed
vanilla_rf = RandomForestClassifier(random_state=seed) 

# Fit the model on the entire data
vanilla_rf.fit(predictions_tune, y_tune);

# Calculate ACC & AUC/ROC on the test set
y_proba = vanilla_rf.predict_proba(predictions_test)[:, 1]
acc = accuracy_score(y_test, y_proba > 0.5)
auc = roc_auc_score(y_test, y_proba)
print(f'Vanilla RF ACC on test set:{acc:.3f}')
print(f'Vanilla RF AUC on test set:{auc:.3f}')

### Number of trees, `n_estimators`, default = 100

Trees in a RF are called _estimators_.
One way to tune our random forest classifer's `n_estimators` hyperparameter is to:
1. iterate over candidate values 
2. instantiate a classifer with current value
3. pass the classifer to `cross_val_score`

In [ ]:
%%time
min_estimators = 50
max_estimators = 500
step_estimators = 50
n_estimators_list = np.arange(min_estimators, max_estimators+1, step_estimators)
cvmeans = []
for n_estimators in n_estimators_list:
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth = 10, max_features='auto', warm_start=True, random_state=seed)
    # Perform 5-fold cross validation and store results
    scores = cross_val_score(estimator=clf, X=predictions_tune, y=y_tune, cv=5)
    cvmeans.append(scores.mean())
    print("n_estimators =", n_estimators,": val_accuracy =",scores.mean())

This can be rather slow, especially if we are using a large number of estimators and/or there are many candidate values we'd like to try.

<div class="discussion"><b>Discussion</b></div>

**Q2.1**: Why do you think cross validation might be slow with random forests?

**Answer:**

 

**Q2.2**: We have this this `predictions_train` dataframe from earlier. Considering what's been said about the tuning data, can you think of any future use for `preductions_train`?

**Answer:**


### OOB Error

A faster option is to use _out-of-bag error (oob)_ to evaluate each candidate hyperparameter value.

Recall that when bootstrapping each estimator in the RF, some observations will not be sampled. Because the estimator wasn't fit on these observations they can be used as a 'free' validation set!

The number of estimators (i.e., trees) needs to be large enough for the $oob$ error to stabilize in its lowest possible value. A good start is 10 times the number of features, however, adjusting other hyperparameters will influence the optimum number of trees (more on that shortly!). 

**Note:** 
* The RF has an `oob_score_` attribute. This is the _oob accuracy_. How would you get the _oob error rate_?
* Instantiating the RF classifer inside the loop at each iteration slows things down. We can instead instantiate the RF _before_ the loop and simply change any parameter values we like from inside the loop using the classifier's `set_params()` method.

In [ ]:
%%time
clf = RandomForestClassifier(warm_start=True, 
                               oob_score=True,
                               min_samples_leaf=40,
                               max_depth = 10,
                               random_state=seed)

error_rate = {}

for i in range(min_estimators, max_estimators + 1):
    clf.set_params(n_estimators=i) 
    clf.fit(X_tune, y_tune)

    # Record the OOB error for each `n_estimators=i` setting.
    oob_error = 1 - clf.oob_score_
    error_rate[i] = oob_error

**Q2.3**: WHat does the `warm_start` parameter do? 

**Answer:**


Now we can plot the $oob$ error of a random forest as a function of the number of trees

In [ ]:
# Generate the "OOB error rate" vs. "n_estimators" plot.
# OOB error rate = num_missclassified/total observations (%)\
xs = []
ys = []
for label, clf_err in error_rate.items():
    xs.append(label)
    ys.append(clf_err)   
plt.plot(xs, ys)
plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.show()

Weevaluated just fit and evaluated 50x the number of models in just twice the time!

### `min_samples_leaf`, default = 1

The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression. We will plot various values of the `min_samples_leaf` with `num_iterators`. 

**Note:** This takes about 2 minutes to run. We can use that time to understand the code! :)

In [ ]:
%%time
# runtime ~2 minutes
from collections import OrderedDict
# instantiate the template classifier
clf = RandomForestClassifier(oob_score=True,
                             max_depth = 10,
                             random_state=seed)

# set some hyperparameter candidates
min_estimators = 100
max_estimators = 650
min_samples_leaf_list = [1, 5]

# Map a label (the value of `min_samples_leaf`) to a list of (model, oob error) tuples.
# error_rate = OrderedDict((label, []) for label in min_samples_leaf_list)
error_rate = {label: [] for label in min_samples_leaf_list}

for min_samples_leaf in min_samples_leaf_list:
    clf.set_params(min_samples_leaf=min_samples_leaf) 
    for i in range(min_estimators, max_estimators + 1):
        clf.set_params(n_estimators=i) 
        clf.fit(X_tune, y_tune)
        # Record the OOB error for each model
        oob_error = 1 - clf.oob_score_
        error_rate[min_samples_leaf].append((i, oob_error))
        # warm_start is complicated if we when looping over
        # other parameters in addition to n_estimators
        clf.set_params(warm_start=True)
    clf.set_params(warm_start=False)

for label, clf_err in error_rate.items():
    xs, ys = zip(*clf_err)
    plt.plot(xs, ys, label=f'min_samples_leaf={label}')

plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.legend(loc="upper right")
plt.show()

We can then look at the lowest error to choose our parameters.

In [ ]:
err = 100
best_num_estimators = 0
for label, clf_err in error_rate.items():
    num_estimators, error = min(clf_err, key=lambda n: (n[1], -n[0]))
    if error<err: err=error; best_num_estimators = num_estimators; best_leaf = label

print(f'Optimum num of estimators: {best_num_estimators} \nmin_samples_leaf: {best_leaf}')

Now we re-train the Random Forest Classifier using the new values for the parameters and calculate the AUC/ROC. Include another parameter, the `max_features`, the number of features to consider when looking for the best split. 

In [ ]:
estimators_rf = RandomForestClassifier(n_estimators= best_num_estimators,
                                    random_state=seed,
                                    oob_score=True,
                                    min_samples_leaf=best_leaf,
                                    max_features='sqrt') 

# Fit the model on the entire data
estimators_rf.fit(predictions_tune, y_tune);

# # Calculate accuracy
# y_pred = estimators_rf.predict(X_test)
# acc = np.round(estimators_rf.score(X_test, y_test),2)
# print(f'Plain RF accuracy on test set:{acc}')

# Calculate AUC/ROC
y_proba = estimators_rf.predict_proba(predictions_test)[:, 1]
estimators_acc = accuracy_score(y_test, y_proba > 0.5)
estimators_auc = np.round(roc_auc_score(y_test, y_proba),2)
print(f'Educated RF ACC on test set:{estimators_acc:.3f}')
print(f'Educated RF AUC on test set:{estimators_auc:.3f}')

Look at the model's parameters with `get_params()`

In [ ]:
estimators_rf.get_params()

### GridSearchCV

After we have some idea of the range of optimum values for the number of trees and maybe a couple of other parameters, and have enough computing power, you may perform an exhaustive search over other parameter values.

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html" target="_blank">GridSearchCV()</a> : Performes exhaustive search over specified parameter values for an estimator.

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# not that warm_start isn't useful if we are not tuning num_estimators
rf = RandomForestClassifier(n_jobs=-1,
                           n_estimators= best_num_estimators,
                           oob_score=True,
                           min_samples_leaf=best_leaf,
                           random_state=seed)

param_dict = {
    'min_samples_split': [2,5,None],
    'max_features': ['sqrt', 'auto']
}

scoring = {'ACC': make_scorer(accuracy_score), 'AUC': 'roc_auc'}

grid_search = GridSearchCV(rf, 
                           param_dict, 
                           scoring=scoring, 
                           refit='ACC', 
                           return_train_score=True, 
                           n_jobs=-1)

results = grid_search.fit(predictions_tune, y_tune)
best_rf = results.best_estimator_
display.display(best_rf.get_params())
# Calculate AUC/ROC
y_proba = best_rf.predict_proba(predictions_test)[:, 1]
grid_acc = accuracy_score(y_test, y_proba > 0.5)
grid_auc = roc_auc_score(y_test, y_proba)
print(f'GridSearchCV RF ACC on test set:{grid_acc:.3f}')
print(f'GridSearchCV RF AUC on test set:{grid_auc:.3f}')

That was pretty quick! But if we wanted to search tune more parameters the number of combinations quickly gets large and we'd really start to notice cross validation slowing us down.

## Skip the CV with ParameterGrid!

[ParameterGrid](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html) creates a generator of dictionaries from your original `param_dict`, one dictionary for each combination of parameters!

Now we can just loop over this generator, using `set_params()` and `oob_score_` without having to be slowed down by cross validation. It's the best of both worlds!

In [ ]:
%%time
# runtime ~1 minute
from sklearn.model_selection import ParameterGrid

param_grid = ParameterGrid(param_dict)

clf = RandomForestClassifier(n_jobs=-1,
                               n_estimators = best_num_estimators,
                               oob_score = True,
                               warm_start=  True,
                               min_samples_leaf=best_leaf,
                               random_state=seed)
error = 100
max_estimators = max([params['n_estimators'] for params in param_grid])
best_params = {}
for params in param_grid:
    clf.set_params(**params)
    clf.fit(predictions_tune, y_tune)
    # Record the OOB error for each model
    oob_error = 1 - clf.oob_score_
    if oob_error < error:
        best_params = clf.get_params()
        if params['n_estimators'] == max_estimators:
            clf.set_params(warm_start=False)
        else:
            clf.set_params(warm_start=True)

In [ ]:
pgrid = clf.set_params(**best_params)
y_proba = pgrid.predict_proba(predictions_test)[:,1]
pgrid_acc = accuracy_score(y_test, y_proba > 0.5)
pgrid_auc = roc_auc_score(y_test, y_proba)
print(f'ParamGrid RF ACC on test set:{pgrid_acc:.3f}')
print(f'ParamGrid RF AUC on test set:{pgrid_auc:.3f}')

## Part 3: Boosting
We wanted to spend a few more words on boosting, since it can sometimes take a second pass to make sense of it.

Boosting is a particular way of building and combing the models in an ensemble. Most ensemble models don't care about the order the models are built in, but in boosting we train a sequence of models where each later model tries to do well on the data points that the current team of models isn't getting right.

In [ ]:
x = np.arange(0, 2*np.pi, 0.1)
y = np.sin(x) + 0.1*np.random.normal(size=x.shape[0])

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
estgb = GradientBoostingRegressor(n_estimators=501, max_depth=1, learning_rate=1)
estgb.fit(x.reshape(-1,1), y);

In [ ]:
import time
display_iters = [0, 1, 2, 3, 4, 5, 6, 10, 20, 50, 100, 200, 400, 500]

def staged_fit_plot(estgb, x, y, display_iters):
    # code from http://nbviewer.jupyter.org/github/pprett/pydata-gbrt-tutorial/blob/master/gbrt-tutorial.ipynb
    fig, ax = plt.subplots(1,2, figsize=(20,10), sharey=True)
    ax[0].plot(x, y, '.');

    sleep_time = 2

    # the predicitions given by staged_predict skips the intial predict-the-mean model, so put it back
    overall_predictions = list(estgb.staged_predict(x.reshape(-1,1)))
    overall_predictions = [np.mean(y)*np.ones_like(x)] + overall_predictions

    # for various points in the run
    for i in display_iters:

        # plot the current overall prediction in the left panel
        cur_overall_prediction = overall_predictions[i]
        ax[0].plot(x, cur_overall_prediction, alpha=0.7, label=str(i), lw=2)
        ax[0].legend()

        # plot the current residuals in the right panel
        resid = y - cur_overall_prediction
        ax[1].cla()
        ax[1].scatter(x,resid, label="Current Residuals")
        ax[1].axhline(0)

        # if early, also plot the model fitted to these residuals
        if i <=5:
            cur_est = estgb.estimators_[i,0]
            cur_prediction = cur_est.predict(x.reshape(-1,1))
            ax[1].plot(x, cur_prediction, color='orange', label="Newest Model")
        else:
            # if late, accalerate
            sleep_time = sleep_time/2
        ax[1].legend()

        # plot
        display.display(fig)
        display.clear_output(wait=True)
        time.sleep(sleep_time)

staged_fit_plot(estgb, x, y, display_iters)

In the left panel, we see an animated version of the slides from class: as we run boosting, the overall model fits (and overfits) the data better and better. In the right panel, we can see the residuals left by the current model, and the model fit to them. At each iteration, $\text{learning_rate}\cdot\text{right_panel_model}$ is added to the latest line in the left panel. You can see how the newest model (right panel) determines how much the overall model (left panel) changes, and why Boosting might be connected to derivatives and gradients.

<div class="exercise"><b>Exercise</b></div>

**Q3.1** What is the effect does the learning rate <1 have on the overall model?

**Answer**:


**Q3.2.** What changes when we use a max_depth of 2 or 3?

**Answer:**


### Tuning a Boosted Model
Now you should try fine tuning a boosted model like the one above using some form of grid search (the `ParameterGrid` method is recommended). Some parameters to play with would be `n_estimators`, `max_depth`, and `learning_rate`. 

**Q3.3** Save your tuned GradientBoostingRegressor object as `gboost` and store its best parameters as `best_params`

**Note:** We've added more data points and more noise to keep things interesting!

**Hints:** 
- You will have to `reshape(-1,1)` the `x` matrix when fitting and predicting, just as in the code above.
- Take a look at the [GradientBoostingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html?highlight=gradientboost#sklearn.ensemble.GradientBoostingRegressor) documentations for more options.
- Refer to the RF GridSearch code above for guidance

In [ ]:
NOISE_SCALE = 0.5
N_POINTS = 2500
x = np.linspace(0, 2*np.pi, N_POINTS)
y = np.sin(x) + NOISE_SCALE*np.random.normal(size=x.shape[0])

def sorted_split(x, y, test_size):
    '''
    We want to shuffle our data when we split.
    But this can result in spaghetti plots later!
    This helper function sorts the results of our splits for us
    '''
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2)
    train_sort_idx = np.argsort(x_train)
    x_train = x_train[train_sort_idx]
    y_train = y_train[train_sort_idx]
    test_sort_idx = np.argsort(x_test)
    x_test = x_test[test_sort_idx]
    y_test = y_test[test_sort_idx]
    return x_train, x_test, y_train, y_test
    
    
x_train, x_test, y_train, y_test = sorted_split(x, y, test_size=.2)
x_train, x_val, y_train, y_val = sorted_split(x_train, y_train, test_size=.2)

In [ ]:
%%time
# your code here


Let's take a look at `best_params` and make sure `gboost` is using them!

In [ ]:
display.display(best_params)
gboost.set_params(**best_params)

If you like, you can view the boosting animation on your own tuned model with the cell below. Watch those residuals shrink!

In [ ]:
n_estimators = best_params['n_estimators']
if n_estimators > max(display_iters):
    display_iters = display_iters + np.arange(max(display_iters), n_estimators+1, 100) 
staged_fit_plot(gboost, x_train, y_train, display_iters=display_iters)

How did you do? Are you overfitting? Below you can view the model predictions on train and test side-by-side as well as the accuracy on all 3 datasets.

**Q3.4** Revisit your gridsearch above and tweak the parameters you are searching over. What's the best test score you can achieve? 

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,10), sharey=True)
for i, (x, y, title) in enumerate(zip([x_train, x_test], [y_train, y_test], ['Train', 'Test'])):
    ax[i].plot(x, y, '.');
    ax[i].plot(x, gboost.predict(x.reshape(-1,1)), alpha=0.7, label=str(i), lw=2)
    ax[i].set_title(f'Gradient Boost on {title} Data')
    ax[i].legend();

In [ ]:
# how high can you get your test score?
print('Train R^2:', gboost.score(x_train.reshape(-1,1), y_train)) 
print('Val   R^2:', gboost.score(x_val.reshape(-1,1), y_val))
print('Test  R^2:', gboost.score(x_test.reshape(-1,1),y_test))

**Please return to the main Zoom room :)**

### Summary of Bagging and Boosting
- Bagging and boosting both build ensemble models in special ways
  - Thus, they each have a specific way of building new models, and a speciefic way of combining them
  - The homework has you explore thier relationship much more deeply